In [1]:
import cudf
import dask.dataframe as dd
import dask
dask.config.set({"dataframe.backend": "cudf"})


price_data = dd.read_csv('/home/edoardocame/Desktop/python_dir/eurgbp1m.csv')
price_data['sma'] = price_data['close'].rolling(60).mean()
price_data['upper_band'] = price_data['sma'] + price_data['close'].rolling(60).std() * 2
price_data['lower_band'] = price_data['sma'] - price_data['close'].rolling(60).std() * 2
price_data.tail()

,timestamp,open,high,low,close,volume,sma,upper_band,lower_band
237678,2024-08-21 23:55:00,0.85204,0.85210,0.85204,0.85209,35.610001,0.852075,0.852183,0.851967
237679,2024-08-21 23:56:00,0.85209,0.85210,0.85204,0.85205,30.100000,0.852075,0.852183,0.851968
237680,2024-08-21 23:57:00,0.85205,0.85205,0.85203,0.85203,13.250000,0.852075,0.852183,0.851968
237681,2024-08-21 23:58:00,0.85203,0.85208,0.85200,0.85207,25.340000,0.852076,0.852183,0.851969
237682,2024-08-21 23:59:00,0.85206,0.85206,0.85199,0.85200,11.280000,0.852075,0.852184,0.851967


In [2]:
#PROTO-LOGIC BEHIND THE BOLLINGER BANDS PANDAS VERSION.

# Initialize positions
price_data['positions'] = 0

# Generate signals
price_data['long_entry'] = (price_data['close'] < price_data['lower_band'])
price_data['long_exit'] = (price_data['close'] > price_data['sma'])
price_data['short_entry'] = (price_data['close'] > price_data['upper_band'])
price_data['short_exit'] = (price_data['close'] < price_data['sma'])

def create_positions(df):
	df.loc[df['long_entry'], 'positions'] = 1
	df.loc[df['long_exit'], 'positions'] = 0
	df.loc[df['short_entry'], 'positions'] = -1
	df.loc[df['short_exit'], 'positions'] = 0
	return df

price_data = price_data.map_partitions(create_positions)

# Forward fill positions
price_data['positions'] = price_data['positions'].fillna(method='ffill').fillna(0)

# Record entry price
price_data['entry_price'] = price_data['close'].where(price_data['positions'].diff() != 0).fillna(method='ffill')

# # Set stop loss percentage
# stop_loss_percentage = 0.05

# def apply_stop_loss(df, stop_loss_percentage):
# 	df.loc[(df['positions'] == 1) & (df['close'] < df['entry_price'] * (1 - stop_loss_percentage)), 'positions'] = 0
# 	df.loc[(df['positions'] == -1) & (df['close'] > df['entry_price'] * (1 + stop_loss_percentage)), 'positions'] = 0
# 	return df

# # Apply stop loss using map_partitions
# price_data = price_data.map_partitions(apply_stop_loss, stop_loss_percentage)

# # Recalculate positions after stop loss
# price_data['positions'] = price_data['positions'].fillna(method='ffill').fillna(0)


# Calculate returns
price_data['market_returns'] = price_data['close'].diff() / price_data['close'].shift(1)
price_data['strategy_returns'] = price_data['market_returns'] * price_data['positions'].shift(1)

# Calculate cumulative returns
price_data['cumulative_equity'] = (1 + price_data['strategy_returns']).cumprod()

In [3]:
price_data.tail()

,timestamp,open,high,low,close,volume,sma,upper_band,lower_band,positions,long_entry,long_exit,short_entry,short_exit,entry_price,market_returns,strategy_returns,cumulative_equity
237678,2024-08-21 23:55:00,0.85204,0.85210,0.85204,0.85209,35.610001,0.852075,0.852183,0.851967,0,False,True,False,False,0.85213,0.000059,0.0,1.062484
237679,2024-08-21 23:56:00,0.85209,0.85210,0.85204,0.85205,30.100000,0.852075,0.852183,0.851968,0,False,False,False,True,0.85213,-0.000047,-0.0,1.062484
237680,2024-08-21 23:57:00,0.85205,0.85205,0.85203,0.85203,13.250000,0.852075,0.852183,0.851968,0,False,False,False,True,0.85213,-0.000023,-0.0,1.062484
237681,2024-08-21 23:58:00,0.85203,0.85208,0.85200,0.85207,25.340000,0.852076,0.852183,0.851969,0,False,False,False,True,0.85213,0.000047,0.0,1.062484
237682,2024-08-21 23:59:00,0.85206,0.85206,0.85199,0.85200,11.280000,0.852075,0.852184,0.851967,0,False,False,False,True,0.85213,-0.000082,-0.0,1.062484
